In [1]:
import requests 
from bs4 import BeautifulSoup

In [2]:
url = "https://www.climatecouncil.org.au/resources/australia-key-role-to-play-clean-energy-reshapes-indo-pacific-relations/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)

In [3]:
soup=BeautifulSoup(response.content, "html.parser")

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-GB"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="en-GB"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="en-GB"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-GB">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://www.climatecouncil.org.au/xmlrpc.php" rel="pingback"/>
  <link href="https://www.climatecouncil.org.au/wp-content/themes/climate-council/assets/dist/img/favicon.png" rel="shortcut icon" type="image/png"/>
  <link href="//cdn.jsdelivr.net/jquery.slick/1.6.0/slick.css" rel="stylesheet" type="text/css">
   <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
    <!-- Google Tag Mana

In [5]:
main_content = soup.find('article')
for element in main_content(['header', 'footer', 'aside', 'script']):
    element.decompose()

In [6]:
text_list = []
stop_text = "The time is right for Australia to accelerate clean energy and associated climate action and secure its economic and strategic advantage in the Indo-Pacific’s clean energy future."
for paragraph in main_content.find_all('p'):
    paragraph_text = paragraph.get_text(separator=' ', strip=False)

    if stop_text in paragraph_text:
        text_list.append(paragraph_text)
        break
    text_list.append(paragraph_text)

In [7]:
article_text = '\n'.join(text_list)

In [8]:
print(article_text)

With the incoming federal government declaring it will make Australia a “renewable superpower”, China dominating clean energy supply chains, and the Russian war in Ukraine disrupting the global energy market, now is a pivotal moment for Australia to shape the future of clean energy within the Indo-Pacific.
A  new report by the Perth USAsia Centre, in collaboration with the Climate Council , recommends five practical actions for Australia to secure its economic and strategic clean energy advantages in the Indo-Pacific :
Published ahead of a major international forum on energy supply chains – the  Sydney Energy Forum  (12-13 July) – the report,  Reenergising Indo-Pacific Relations: Australia’s Clean Energy Opportunity ,  explains that the Indo-Pacific sits at the heart of the global shift from fossil fuels to clean energy systems, the ramifications for Australia’s economic and strategic interests, and the leading role that our country could play.
Global warming is a significant security 

In [9]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [14]:
import nltk
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/loogyee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
def preprocess_text(text):
    # Lowercase the entire text
    text = text.lower()
    
    # Initialize the lemmatizer and stemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # Lemmatization and Stemming
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    stemmed_words = [stemmer.stem(word) for word in words]
    
    processed_text = ' '.join(lemmatized_words)

    return processed_text

In [13]:
def preprocess_text(text):
    # Lowercase the entire text
    text = text.lower()
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = re.findall(r'\w+', text)  # Find all alphanumeric words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize words
    
    # Join lemmatized words back into a single string
    cleaned_text = ' '.join(lemmatized_words)
    
    return cleaned_text

In [17]:
cleaned_text = preprocess_text(article_text)

In [18]:
print(cleaned_text)

with the incoming federal government declaring it will make australia a “renewable superpower”, china dominating clean energy supply chains, and the russian war in ukraine disrupting the global energy market, now is a pivotal moment for australia to shape the future of clean energy within the indo-pacific. a new report by the perth usasia centre, in collaboration with the climate council , recommends five practical action for australia to secure it economic and strategic clean energy advantage in the indo-pacific : published ahead of a major international forum on energy supply chain – the sydney energy forum (12-13 july) – the report, reenergising indo-pacific relations: australia’s clean energy opportunity , explains that the indo-pacific sits at the heart of the global shift from fossil fuel to clean energy systems, the ramification for australia’s economic and strategic interests, and the leading role that our country could play. global warming is a significant security threat for 

In [19]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0.tar.gz

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0.tar.gz (12.8 MB)
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [22]:
doc = nlp(cleaned_text)
unique_entities = set()

for ent in doc.ents:
    unique_entities.add((ent.text, ent.label_))
for entity, label in unique_entities:
    print(f"{entity} - {label}")

12-13 july - DATE
five - CARDINAL
james bowen - PERSON
dr wesley - PERSON
australia - GPE
the perth usasia centre - ORG
ukraine - GPE
china - GPE
australian - NORP
korea - GPE
japan - GPE
the russian war - EVENT


In [23]:
important_deps = ["nsubj", "dobj", "ROOT", "xcomp", "ccomp", "conj", "amod"]
print("\nDependency Parsing (Relationships):")
for token in doc:
    if token.dep_ in important_deps:
        print(f"{token.text} --> {token.dep_} --> {token.head.text}")


Dependency Parsing (Relationships):
incoming --> amod --> government
federal --> amod --> government
government --> nsubj --> declaring
it --> nsubj --> make
make --> ccomp --> declaring
australia --> nsubj --> superpower
renewable --> amod --> superpower
superpower --> ccomp --> make
china --> nsubj --> is
clean --> amod --> chains
chains --> dobj --> dominating
russian --> amod --> war
war --> conj --> china
global --> amod --> market
market --> dobj --> disrupting
is --> ROOT --> is
pivotal --> amod --> moment
australia --> nsubj --> shape
future --> dobj --> shape
clean --> amod --> energy
new --> amod --> report
report --> nsubj --> recommends
recommends --> ccomp --> explains
practical --> amod --> action
action --> dobj --> recommends
australia --> nsubj --> secure
it --> dobj --> secure
economic --> amod --> advantage
strategic --> conj --> economic
clean --> amod --> energy
advantage --> dobj --> secure
major --> amod --> forum
international --> amod --> forum
reenergising --

In [24]:
from spacy import displacy

In [25]:
sentence_spans = list(doc.sents)
displacy.render(sentence_spans, style='dep', jupyter=True, options={'compact': True})

In [26]:
sentence_spans = list(doc.sents)
displacy.render(sentence_spans, style="ent", jupyter=True)

/Users/loogyee/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [33]:
relationship_keywords = {'collaboration', 'cooperation', 'partner', 'engage', 
                         'support', 'fund', 'invest', 'develop', 
                         'supply', 'promote', 'transition', 'export', 'agreement', 'interest'
                         'share', 'enhance'}

In [34]:
entity1_list = []
relationship_list = []
entity2_list = []
date_list = []

In [35]:
for sent in doc.sents:
    # Extract named entities (countries, organizations)
    entities = [ent for ent in sent.ents if ent.label_ in ['GPE', 'ORG']]
    
    # Look for verbs or specific relationship nouns in the sentence
    verbs = [token for token in sent if token.pos_ == "VERB"]
    keywords_in_sentence = [token.text for token in sent if token.lemma_ in relationship_keywords]
    
    # Extract dates
    dates = [ent.text for ent in sent.ents if ent.label_ == "DATE"]
    
    # If we find two entities and a relationship keyword, we assume a relationship
    if len(entities) >= 2 and keywords_in_sentence:
        entity1_list.append(entities[0].text)
        
        # Concatenate the relationship noun and verb (if available) into one string
        relationship_info = keywords_in_sentence[0]
        if verbs:
            relationship_info += " (verb: " + verbs[0].lemma_ + ")"
        
        relationship_list.append(relationship_info)  # store concatenated info
        entity2_list.append(entities[1].text)
        date_list.append(dates[0] if dates else "n/a")

In [36]:
import pandas as pd

In [37]:
# DataFrame
df = pd.DataFrame({
    "Entity1": entity1_list,
    "Relationship": relationship_list,
    "Entity2": entity2_list,
    "Date": date_list
})

In [38]:
df

,Entity1,Relationship,Entity2,Date
0,china,supply (verb: declare),ukraine,n/a
1,the perth usasia centre,collaboration (verb: recommend),australia,12-13 july
2,japan,exports (verb: continue),korea,n/a


In [ ]:
# Export to csv
df.to_csv("international_cooperation_renewable_energy_1.csv", index=False)